## prepare dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!gdown --id 1wCNiI0_V9sBO6srQuXftx0LxBsdHYBkt

In [ ]:
!unzip /content/googleNet.zip

In [ ]:
import glob
import imageio
import os
import cv2

for folder in glob.glob('/content/googleNet/*'):
  for subfolder in glob.glob(folder+'/*'):
    for image in glob.glob(subfolder+'/*'):
      
      try:
        im = imageio.imread(image)

        if im.shape[2] != 3 :
          print(image)
          os.remove(image)

      except:
        print(image)
        os.remove(image)

In [ ]:
!rm -rf /content/googleNet/Train/Fire/.ipynb_checkpoints/
!rm -rf /content/googleNet/Validation/.ipynb_checkpoints/
!rm -rf /content/googleNet/Train/.ipynb_checkpoints/

In [ ]:
import tensorflow as tf

img_height = 224
img_width = 224
batch_size = 64

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/googleNet/Train",
    seed=123,
    shuffle=True,
    image_size=(img_height, img_width),
    label_mode = "categorical",
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/googleNet/Validation",
    seed=123,
    shuffle=True,
    image_size=(img_height, img_width),
    label_mode="categorical",
    batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

### requirements for installation 

In [ ]:
!pip install image-classifiers==0.2.2

In [ ]:
!pip install git+https://github.com/qubvel/classification_models.git

## Loading model with imagenet weights:

In [ ]:
# from classification_models.keras import Classifiers

# for tensorflow.keras
from classification_models.tfkeras import Classifiers

ResNet18, preprocess_input = Classifiers.get('resnet18')
model = ResNet18((224, 224, 3), weights='imagenet')

model.summary()

## Model fine-tuning example:

In [ ]:
transfer_model = ResNet18(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
for layer in transfer_model.layers[:-1]: 
    layer.trainable = True

    transfer_model.summary()

# build model

In [ ]:
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.models import Model


x = GlobalAveragePooling2D()(transfer_model.output)
x = Dense(2, activation = 'relu')(x)
mymodel = Dense(2, activation='softmax')(x)

my_finetunResnet = Model(transfer_model.input, mymodel)

my_finetunResnet.summary()

## compile model

In [ ]:
import tensorflow as tf
my_finetunResnet.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001), loss=tf.keras.losses.CategoricalCrossentropy() , metrics=['Accuracy', 'Precision', 'Recall'])

## define Callback

In [ ]:
myCallback = tf.keras.callbacks.ModelCheckpoint(
    "/content/drive/MyDrive/resnet18_v6", monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
)

## load saved model: 
This line only applies to saved models

In [ ]:
import tensorflow as tf
my_finetunResnet = tf.keras.models.load_model("/content/drive/MyDrive/resnet18_v5") 

## Train Model


In [ ]:
history = my_finetunResnet.fit(
    train_ds,
    epochs=300,
    validation_data=val_ds,
    callbacks=[myCallback]
)

## Convert to ONNX

In [ ]:
!pip install tf2onnx

In [ ]:
import onnx
import tf2onnx

onnx_model, _ = tf2onnx.convert.from_keras(myModel)
onnx.save(onnx_model, "/content/drive/MyDrive/resnet18_v5.onnx")